In [ ]:
import json
import os
import requests
from datetime import datetime, timedelta
from dotenv import load_dotenv

In [ ]:
def generateToken():
    cwd = os.getcwd()
    dotenv_path = os.path.join(cwd,'filepath')
    load_dotenv(dotenv_path)
    time_generated_str = os.getenv("TIME_GENERATED")
    print(time_generated_str)
    TimeGenerated = datetime.fromisoformat(time_generated_str)
    if TimeGenerated + timedelta(minutes=60) < datetime.now():
        token_url = os.getenv("ISSUER_URL")+"/v1/token"
        print(token_url)
        client_id = os.getenv("CLIENT_ID")
        client_secret = os.getenv("CLIENT_SECRET")
        scope = os.getenv("SCOPE")
        # Define the payload
        payload = {
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
            "scope": scope
        }
        # Make the POST request
        response = requests.post(token_url, data=payload, headers={"Content-Type": "application/x-www-form-urlencoded"})
        print(response.json())
        repsonse_json = response.json()
        token = repsonse_json["access_token"]
        TimeGenerated = datetime.now()
        # Update the .env file with the new token
        with open('C:/Users/AK57630/auth.env', 'r') as file:
            lines = file.readlines()
        with open('.env', 'w') as file:
            for line in lines:
                if line.startswith("OPENAI_API_KEY="):
                    file.write(f"OPENAI_API_KEY={token}\n")
                elif line.startswith("TIME_GENERATED="):
                    file.write(f"TIME_GENERATED={TimeGenerated}\n")
                else:
                    file.write(line)    
        os.environ.pop("OPENAI_API_KEY", None)                
        load_dotenv()
        print(os.getenv("OPENAI_API_KEY"))   

In [ ]:
generateToken()

In [ ]:
from langchain_openai import ChatOpenAI

header_name = os.getenv('AI_GATEWAY_HEADER_NAME')
header_value = os.getenv('AI_GATEWAY_REGISTRATION_ID')
headers = {
    header_name: header_value
    }
chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0,default_headers=headers)

In [ ]:
#!pip install PyPDF2
import PyPDF2

In [ ]:
#Open the file 
file_name = os.getenv("FILEPATH")
with open(file_name,'rb') as file:
    reader=PyPDF2.PdfReader(file)
    #Extract text from all pages
    study_material=""
    for page in reader.pages:
        study_material+=page.extract_text()
    #Now 'study_material' contains the text from PDF
    print(study_material)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Chain 1: Summarize the text
summary_prompt = """
       Act as a study assistant, given the following study material,
       generate a summary of the study material into key bullet points.

       Study Material:
       {study_material}
"""
prompt_template1 = ChatPromptTemplate.from_template(summary_prompt)
llm_chain1 = (prompt_template1
                  |
              chatgpt
                  |
              StrOutputParser())

In [ ]:
# Chain 2: Translate Customer Message to English
prompt2 = """
  Act as a study assistant,
  For the summary message delimited below by triple backticks,
  Generate 5 multiple choice questions and also mention the correct answer at the end of each questions
 Summary:
  ```{summary}```
"""
prompt_template2 = ChatPromptTemplate.from_template(prompt2)
llm_chain2 = (prompt_template2
                  |
              chatgpt
                  |
              StrOutputParser())

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

final_chain = (
    RunnablePassthrough.assign(summary=llm_chain1)
      |
    RunnablePassthrough.assign(quiz_questions=llm_chain2)
)

In [ ]:
response = final_chain.invoke({'study_material': study_material})
response

In [ ]:
from IPython.display import Markdown, display

display(Markdown(response["summary"]))

In [ ]:
from IPython.display import Markdown, display

display(Markdown(response["quiz_questions"]))